<a href="https://colab.research.google.com/github/Kavyapm1960/project/blob/main/EN_FR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

def load_data(filename):
    with open(filename, encoding='utf-8') as file:
        text = file.read()
    return text.split('\n')

def tokenize(sentences):
    tokenizer = Tokenizer(char_level=False)
    tokenizer.fit_on_texts(sentences)
    return tokenizer, tokenizer.texts_to_sequences(sentences)

def pad(sequences, maxlen=None):
    return pad_sequences(sequences, padding='post', maxlen=maxlen)

english_sentences = load_data('/content/drive/MyDrive/small_vocab_en.txt')
french_sentences = load_data('/content/drive/MyDrive/small_vocab_fr.txt')

english_tokenizer, english_tokenized = tokenize(english_sentences)
french_tokenizer, french_tokenized = tokenize(french_sentences)

max_english_length = max(len(sentence) for sentence in english_tokenized)
max_french_length = max(len(sentence) for sentence in french_tokenized)

english_padded = pad(english_tokenized, maxlen=max_english_length)
french_padded = pad(french_tokenized, maxlen=max_french_length)

encoder_input_data = np.array(english_padded)
decoder_input_data = np.array(french_padded)
decoder_output_data = np.zeros_like(decoder_input_data)
decoder_output_data[:, :-1] = decoder_input_data[:, 1:]

embedding_size = 256
lstm_units = 512
dropout_rate = 0.5

num_encoder_tokens = len(english_tokenizer.word_index) + 1
num_decoder_tokens = len(french_tokenizer.word_index) + 1

encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(num_encoder_tokens, embedding_size)(encoder_inputs)
encoder_lstm = LSTM(lstm_units, return_state=True, dropout=dropout_rate)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None,))
decoder_embedding_layer = Embedding(num_decoder_tokens, embedding_size)
decoder_embedding = decoder_embedding_layer(decoder_inputs)
decoder_lstm = LSTM(lstm_units, return_sequences=True, return_state=True, dropout=dropout_rate)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

decoder_output_one_hot = np.zeros(
    (len(french_sentences), max_french_length, num_decoder_tokens),
    dtype='float32'
)

for i, sentence in enumerate(decoder_output_data):
    for t, word_idx in enumerate(sentence):
        if word_idx > 0:
            decoder_output_one_hot[i, t, word_idx] = 1

model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_output_one_hot,
    batch_size=512,
    epochs=200,
    validation_split=0.2
)

encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(lstm_units,))
decoder_state_input_c = Input(shape=(lstm_units,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_embedding_output = decoder_embedding_layer(decoder_inputs)

decoder_outputs, state_h, state_c = decoder_lstm(decoder_embedding_output, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)

reverse_french_tokenizer = {i: word for word, i in french_tokenizer.word_index.items()}


Epoch 1/200
216/216 [==============================] - 27s 91ms/step - loss: 2.1610 - accuracy: 0.0913 - val_loss: 1.9537 - val_accuracy: 0.1212
Epoch 2/200
216/216 [==============================] - 16s 75ms/step - loss: 1.8323 - accuracy: 0.1321 - val_loss: 1.6791 - val_accuracy: 0.1585
Epoch 3/200
216/216 [==============================] - 16s 75ms/step - loss: 1.5969 - accuracy: 0.1786 - val_loss: 1.5082 - val_accuracy: 0.2155
Epoch 4/200
216/216 [==============================] - 15s 70ms/step - loss: 1.4443 - accuracy: 0.2151 - val_loss: 1.3733 - val_accuracy: 0.2272
Epoch 5/200
216/216 [==============================] - 16s 76ms/step - loss: 1.3152 - accuracy: 0.2479 - val_loss: 1.2360 - val_accuracy: 0.2646
Epoch 6/200
216/216 [==============================] - 15s 71ms/step - loss: 1.2022 - accuracy: 0.2700 - val_loss: 1.1694 - val_accuracy: 0.2788
Epoch 7/200
216/216 [==============================] - 16s 74ms/step - loss: 1.1135 - accuracy: 0.2861 - val_loss: 1.0663 - val_ac